In [1]:
#importing required Libraries
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import numpy as np


In [2]:
#creating empty list 
movie_name = []
year = []
rating = []
time = []
metascore = []
votes = []
gross = []


In [3]:
movie_no=1
while movie_no <= 201:
    
    url = "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start='+movie_no+'&ref_=adv_nxt"
    response = requests.get(url)
    movie_no += 50
    soup = BeautifulSoup(response.content, 'html.parser')
    movie_data = soup.findAll('div',attrs ={'class': 'lister-item mode-advanced'})
    for store in movie_data:    
        name = store.h3.a.text
        movie_name.append(name)
        
        
        year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
        year.append(year_of_release)
    
        runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
        time.append(runtime)

        rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
        rating.append(rate)

        meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '^^^^^^'
        metascore.append(meta)
        
        value = store.find_all('span', attrs = {'name': 'nv'})

        vote = value[0].text
        votes.append(vote)

        grosses = value[1].text if len(value) >1 else '*****'
        gross.append(grosses)

In [4]:
np.count_nonzero(movie_name)

250

In [5]:
#creating a dataframe using pandas library
movie_DF = pd.DataFrame({'Name of movie': movie_name, 'Year of relase': year, 'Watchtime': time, 'Movie Rating': rating, 'Metascore': metascore, 'Votes': votes, 'Gross collection': gross})


In [6]:
movie_DF.head(250)

,Name of movie,Year of relase,Watchtime,Movie Rating,Metascore,Votes,Gross collection
0,The Shawshank Redemption,1994,142,9.3,81,"2,701,019",$28.34M
1,The Godfather,1972,175,9.2,100,"1,874,962",$134.97M
2,The Dark Knight,2008,152,9.0,84,"2,674,782",$534.86M
3,Schindler's List,1993,195,9.0,95,"1,365,382",$96.90M
4,The Lord of the Rings: The Return of the King,2003,201,9.0,94,"1,859,884",$377.85M
...,...,...,...,...,...,...,...
245,The Pianist,2002,150,8.5,85,"840,491",$32.57M
246,The Intouchables,2011,112,8.5,57,"867,008",$13.18M
247,Casablanca,1942,102,8.5,100,"576,064",$1.02M
248,Psycho,1960,109,8.5,97,"677,364",$32.00M


In [11]:
from sqlalchemy import inspect, create_engine
from sqlalchemy import event

alchemy_driver = 'postgresql+psycopg2'
postgres_user = 'postgres'
postgres_password = '123456789'
postgres_host = '127.0.0.1'
postgres_db = 'imdb_data'
postgres_schema = 'public'


postgres_conn = create_engine(f'{alchemy_driver}://{postgres_user}:{postgres_password}@{postgres_host}:5432/{postgres_db}')
print("Postgres connection set")

movie_DF.to_sql(name='imdb_movies', con=postgres_conn, schema=postgres_schema, if_exists='replace', index=False)



Postgres connection set


250